# LAB | Ensemble Methods

**Load the data**

In this challenge, we will be working with the same Spaceship Titanic data, like the previous Lab. The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

In this Lab, you should try different ensemble methods in order to see if can obtain a better model than before. In order to do a fair comparison, you should perform the same feature scaling, engineering applied in previous Lab.

In [70]:
#Libraries
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [7]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [12]:
spaceship.dropna(inplace= True)
spaceship['Cabin'] = spaceship['Cabin'].str.split('/').str[0]
spaceship =spaceship.drop(columns = ["PassengerId","Name"])


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [14]:
spaceship_cat = spaceship[["HomePlanet", "CryoSleep","Destination","Cabin","VIP"]]
num_f = spaceship.drop(columns=["HomePlanet", "CryoSleep","Destination","Cabin","VIP","Transported"])
non_numerical = pd.get_dummies(spaceship_cat, drop_first = True).astype(int)

**Perform Train Test Split**

In [54]:
features = pd.concat([num_f,non_numerical], axis = 1)
target = spaceship["Transported"]
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.20, random_state=0)
normalizer =MinMaxScaler()
normalizer.fit(X_train)
X_train_norm = normalizer.transform(X_train)
X_test_norm = normalizer.transform(X_test)

In [42]:
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

LinearRegression()

In [44]:
pred = lin_reg.predict(X_test)

In [45]:
lin_reg.score(X_test, y_test)

0.3455953983584452

**Model Selection** - now you will try to apply different ensemble methods in order to get a better model

- Bagging and Pasting

In [73]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier,AdaBoostClassifier
bagging_reg = BaggingClassifier(DecisionTreeClassifier(max_depth=20),
                               n_estimators=100,
                               max_samples = 1000)

In [74]:
bagging_reg.fit(X_train, y_train)

BaggingClassifier(estimator=DecisionTreeClassifier(max_depth=20),
                  max_samples=1000, n_estimators=100)

In [75]:
bagging_reg.score(X_test, y_test)

0.7821482602118003

- Random Forests

In [76]:
forest = RandomForestClassifier(n_estimators=100,
                             max_depth=20)

In [77]:
forest.fit(X_train, y_train)

RandomForestClassifier(max_depth=20)

In [78]:
forest.score(X_test, y_test)

0.7821482602118003

- Gradient Boosting

In [81]:
from sklearn.ensemble import GradientBoostingClassifier
gb_reg = GradientBoostingClassifier(max_depth=20,
                                   n_estimators=100)

In [82]:
gb_reg.fit(X_train, y_train)

GradientBoostingClassifier(max_depth=20)

In [83]:
pred = gb_reg.predict(X_test)

In [84]:
gb_reg.score(X_test, y_test)

0.7829046898638427

- Adaptive Boosting

In [91]:
ada_reg = AdaBoostClassifier(DecisionTreeClassifier(max_depth=20),
                            n_estimators=100)

In [92]:
ada_reg.fit(X_train_norm, y_train)

AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=20),
                   n_estimators=100)

In [93]:
pred = ada_reg.predict(X_test_norm)
pred

array([ True,  True,  True, ...,  True,  True,  True])

In [94]:
ada_reg.score(X_test_norm, y_test)

0.7692889561270801

Which model is the best and why?

they are all bad